<a href="https://colab.research.google.com/github/vitthal-bhandari/NER/blob/master/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
tf.__version__

'2.7.0'

In [ ]:
tfds.__version__

'4.0.1'

In [ ]:
#check gpu information
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Jan  1 18:21:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#check VM runtime information
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
#download data
!wget https://gmb.let.rug.nl/releases/gmb-2.2.0.zip
!unzip gmb-2.2.0.zip

Streaming output truncated to the last 5000 lines.
  inflating: gmb-2.2.0/data/p00/d0294/en.drs.xml  
  inflating: gmb-2.2.0/data/p00/d0294/en.drg  
   creating: gmb-2.2.0/data/p00/d0692/
  inflating: gmb-2.2.0/data/p00/d0692/en.met  
  inflating: gmb-2.2.0/data/p00/d0692/en.raw  
  inflating: gmb-2.2.0/data/p00/d0692/en.tok.off  
  inflating: gmb-2.2.0/data/p00/d0692/en.tok.iob  
  inflating: gmb-2.2.0/data/p00/d0692/en.tags  
  inflating: gmb-2.2.0/data/p00/d0692/en.der.xml  
  inflating: gmb-2.2.0/data/p00/d0692/en.drs.xml  
  inflating: gmb-2.2.0/data/p00/d0692/en.drg  
   creating: gmb-2.2.0/data/p00/d0158/
  inflating: gmb-2.2.0/data/p00/d0158/en.met  
  inflating: gmb-2.2.0/data/p00/d0158/en.raw  
  inflating: gmb-2.2.0/data/p00/d0158/en.tok.off  
  inflating: gmb-2.2.0/data/p00/d0158/en.tok.iob  
  inflating: gmb-2.2.0/data/p00/d0158/en.tags  
  inflating: gmb-2.2.0/data/p00/d0158/en.der.xml  
  inflating: gmb-2.2.0/data/p00/d0158/en.drs.xml  
  inflating: gmb-2.2.0/data/p00/d0

In [ ]:
import os
data_root = './gmb-2.2.0/data/'
fnames = []
for root, dirs, files in os.walk(data_root):
  for filename in files:
    if filename.endswith(".tags"):
      fnames.append(os.path.join(root, filename))
fnames[:2]

['./gmb-2.2.0/data/p03/d0611/en.tags', './gmb-2.2.0/data/p03/d0243/en.tags']

In [ ]:
!mkdir ner

In [ ]:
#strip sub categories from the NER tags
import csv
import collections

ner_tags = collections.Counter()
iob_tags = collections.Counter()

def strip_ner_subcat(tag):
  # NER tags are of form {cat}-{subcat}
  return tag.split("-")[0]

In [ ]:
def iob_format(ners):
  # converts IO tags into IOB format
  # input is a sequence of IO NER tokens
  # convert this: O, PERSON, PERSON, O, O, LOCATION, O
  # into: O, B-PERSON, I-PERSON, O, O, B-LOCATION, O
  iob_tokens = []
  for idx, token in enumerate(ners):
    if token != 'O': # !other
      if idx == 0:
        token = "B-" + token #start of sentence
      elif ners[idx-1] == token:
        token = "I-" + token # continues
      else:
        token = "B-" + token
    iob_tokens.append(token)
    iob_tags[token] += 1
  return iob_tokens

In [ ]:
total_sentences=0
outfiles=[]
for idx, file in enumerate(fnames):
  with open(file, 'rb') as content:
    data=content.read().decode('utf-8').strip()
    sentences=data.split("\n\n")
    print(idx, file, len(sentences))
    total_sentences+=len(sentences)

    with open("./ner/"+str(idx)+"-"+os.path.basename(file), 'w') as outfile:
      outfiles.append("./ner/"+str(idx)+"-"+os.path.basename(file))
      writer=csv.writer(outfile)

      for sentence in sentences:
        toks=sentence.split('\n')
        words, pos, ner = [], [], []

        for tok in toks:
          t=tok.split("\t")
          words.append(t[0])
          pos.append(t[1])
          ner_tags[t[3]]+=1
          ner.append(strip_ner_subcat(t[3]))
        writer.writerow([" ".join(words),
                         " ".join(iob_format(ner)),
                         " ".join(pos)])

Streaming output truncated to the last 5000 lines.
5000 ./gmb-2.2.0/data/p55/d0651/en.tags 4
5001 ./gmb-2.2.0/data/p55/d0120/en.tags 5
5002 ./gmb-2.2.0/data/p55/d0618/en.tags 5
5003 ./gmb-2.2.0/data/p55/d0182/en.tags 8
5004 ./gmb-2.2.0/data/p55/d0485/en.tags 5
5005 ./gmb-2.2.0/data/p55/d0240/en.tags 7
5006 ./gmb-2.2.0/data/p55/d0520/en.tags 8
5007 ./gmb-2.2.0/data/p55/d0016/en.tags 6
5008 ./gmb-2.2.0/data/p55/d0437/en.tags 7
5009 ./gmb-2.2.0/data/p55/d0296/en.tags 6
5010 ./gmb-2.2.0/data/p55/d0051/en.tags 6
5011 ./gmb-2.2.0/data/p55/d0497/en.tags 8
5012 ./gmb-2.2.0/data/p55/d0319/en.tags 6
5013 ./gmb-2.2.0/data/p55/d0005/en.tags 4
5014 ./gmb-2.2.0/data/p55/d0262/en.tags 6
5015 ./gmb-2.2.0/data/p55/d0093/en.tags 6
5016 ./gmb-2.2.0/data/p55/d0528/en.tags 7
5017 ./gmb-2.2.0/data/p55/d0449/en.tags 7
5018 ./gmb-2.2.0/data/p55/d0564/en.tags 6
5019 ./gmb-2.2.0/data/p55/d0342/en.tags 5
5020 ./gmb-2.2.0/data/p55/d0112/en.tags 6
5021 ./gmb-2.2.0/data/p55/d0266/en.tags 7
5022 ./gmb-2.2.0/data/p55

In [ ]:
print("total number of sentences: ", total_sentences)

total number of sentences:  62010


In [ ]:
print(ner_tags)
print(iob_tags)

Counter({'O': 1146068, 'geo-nam': 58388, 'org-nam': 48034, 'per-nam': 23790, 'gpe-nam': 20680, 'tim-dat': 12786, 'tim-dow': 11404, 'per-tit': 9800, 'per-fam': 8152, 'tim-yoc': 5290, 'tim-moy': 4262, 'per-giv': 2413, 'tim-clo': 891, 'art-nam': 866, 'eve-nam': 602, 'nat-nam': 300, 'tim-nam': 146, 'eve-ord': 107, 'per-ini': 60, 'org-leg': 60, 'per-ord': 38, 'tim-dom': 10, 'art-add': 1, 'per-mid': 1})
Counter({'O': 1146068, 'B-geo': 48876, 'B-tim': 26296, 'B-org': 26195, 'I-per': 22270, 'B-per': 21984, 'I-org': 21899, 'B-gpe': 20436, 'I-geo': 9512, 'I-tim': 8493, 'B-art': 503, 'B-eve': 391, 'I-art': 364, 'I-eve': 318, 'I-gpe': 244, 'B-nat': 238, 'I-nat': 62})


# Normalizing and Vectorizing the data

In [ ]:
import glob
import pandas as pd

# could use `outfiles` param as well
files = glob.glob("./ner/*.tags")

data_pd = pd.concat([pd.read_csv(f, header=None, 
                                 names=["text", "label", "pos"]) 
                for f in files], ignore_index = True)

In [ ]:
data_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62010 entries, 0 to 62009
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    62010 non-null  object
 1   label   62010 non-null  object
 2   pos     62010 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [ ]:
### Keras tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
text_tok = Tokenizer(filters='[\\]^\t\n', lower=False,
                     split=' ', oov_token='<OOV>')

pos_tok = Tokenizer(filters='\t\n', lower=False,
                    split=' ', oov_token='<OOV>')

ner_tok = Tokenizer(filters='\t\n', lower=False,
                    split=' ', oov_token='<OOV>')

In [ ]:
text_tok.fit_on_texts(data_pd['text'])
pos_tok.fit_on_texts(data_pd['pos'])
ner_tok.fit_on_texts(data_pd['label'])

In [ ]:
ner_config = ner_tok.get_config()
text_config = text_tok.get_config()

In [ ]:
print(ner_config)

{'num_words': None, 'filters': '\t\n', 'lower': False, 'split': ' ', 'char_level': False, 'oov_token': '<OOV>', 'document_count': 62010, 'word_counts': '{"B-org": 26195, "I-org": 21899, "O": 1146068, "B-tim": 26296, "B-per": 21984, "I-per": 22270, "B-geo": 48876, "I-tim": 8493, "B-gpe": 20436, "I-geo": 9512, "B-eve": 391, "I-gpe": 244, "B-art": 503, "I-art": 364, "I-eve": 318, "B-nat": 238, "I-nat": 62}', 'word_docs': '{"B-org": 20478, "I-org": 11011, "O": 61999, "B-tim": 22345, "I-per": 13805, "B-per": 17499, "B-geo": 31660, "I-tim": 5526, "B-gpe": 16565, "I-geo": 7738, "B-eve": 361, "I-gpe": 224, "B-art": 425, "I-art": 207, "I-eve": 201, "B-nat": 211, "I-nat": 50}', 'index_docs': '{"5": 20478, "8": 11011, "2": 61999, "4": 22345, "6": 13805, "7": 17499, "3": 31660, "11": 5526, "9": 16565, "10": 7738, "13": 361, "16": 224, "12": 425, "14": 207, "15": 201, "17": 211, "18": 50}', 'index_word': '{"1": "<OOV>", "2": "O", "3": "B-geo", "4": "B-tim", "5": "B-org", "6": "I-per", "7": "B-per",

In [ ]:
text_vocab = eval(text_config['index_word'])
ner_vocab = eval(ner_config['index_word'])

print("Unique words in vocab:", len(text_vocab))
print("Unique NER tags in vocab:", len(ner_vocab))

Unique words in vocab: 39422
Unique NER tags in vocab: 18


In [ ]:
x_tok = text_tok.texts_to_sequences(data_pd['text'])
y_tok = ner_tok.texts_to_sequences(data_pd['label'])

In [ ]:
print(text_tok.sequences_to_texts([x_tok[1]]), data_pd['text'][1])
print(ner_tok.sequences_to_texts([y_tok[1]]), data_pd['label'][1])

['Sony Columbia Acquisition Corp. , formed for the Columbia deal , will formally take ownership of the movie studio later this month , a spokesman said .'] Sony Columbia Acquisition Corp. , formed for the Columbia deal , will formally take ownership of the movie studio later this month , a spokesman said .
['B-org I-org I-org I-org O O O O O O O O O O O O O O O B-tim O O O O O O O'] B-org I-org I-org I-org O O O O O O O O O O O O O O O B-tim O O O O O O O


In [ ]:
# now, pad seqences to a maximum length
from tensorflow.keras.preprocessing import sequence

max_len = 50

x_pad = sequence.pad_sequences(x_tok, padding='post',
                              maxlen=max_len)
y_pad = sequence.pad_sequences(y_tok, padding='post',
                              maxlen=max_len)

In [ ]:
print(x_pad.shape, y_pad.shape)

(62010, 50) (62010, 50)


In [ ]:
text_tok.sequences_to_texts([x_pad[1]])

['Sony Columbia Acquisition Corp. , formed for the Columbia deal , will formally take ownership of the movie studio later this month , a spokesman said . <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV>']

In [ ]:
ner_tok.sequences_to_texts([y_pad[1]])

['B-org I-org I-org I-org O O O O O O O O O O O O O O O B-tim O O O O O O O <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV>']

In [ ]:
num_classes = len(ner_vocab)+1

Y = tf.keras.utils.to_categorical(y_pad, num_classes=num_classes)
Y.shape

(62010, 50, 19)

In [ ]:
# Length of the vocabulary
vocab_size = len(text_vocab) + 1
# The embedding dimension
embedding_dim = 64
# Number of RNN units
rnn_units = 100
#batch size
BATCH_SIZE=90
# num of NER classes
num_classes = len(ner_vocab)+1

In [ ]:
#instantiate model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, TimeDistributed, Dense
dropout=0.2
def build_model_bilstm(vocab_size, embedding_dim, rnn_units, batch_size, classes):
  model = tf.keras.Sequential([
                               Embedding(vocab_size, embedding_dim, mask_zero=True,
                              batch_input_shape=[batch_size, None]),
                               Bidirectional(LSTM(units=rnn_units,
                           return_sequences=True,
                           dropout=dropout,  
                           kernel_initializer=tf.keras.initializers.he_normal())),
                           TimeDistributed(Dense(rnn_units, activation='relu')),
    Dense(num_classes, activation="softmax")
      ])

  
  return model

In [ ]:
model = build_model_bilstm(
                        vocab_size = vocab_size,
                        embedding_dim=embedding_dim,
                        rnn_units=rnn_units,
                        batch_size=BATCH_SIZE,
                        classes=num_classes)
model.summary()
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (90, None, 64)            2523072   
                                                                 
 bidirectional (Bidirectiona  (90, None, 200)          132000    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (90, None, 100)          20100     
 ibuted)                                                         
                                                                 
 dense_1 (Dense)             (90, None, 19)            1919      
                                                                 
Total params: 2,677,091
Trainable params: 2,677,091
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# to enable TensorFlow to process sentences properly
X = x_pad
# create training and testing splits
total_sentences = 62010
test_size = round(total_sentences / BATCH_SIZE * 0.2)
X_train = X[BATCH_SIZE*test_size:]
Y_train = Y[BATCH_SIZE*test_size:]
X_test = X[0:BATCH_SIZE*test_size]
Y_test = Y[0:BATCH_SIZE*test_size]

In [ ]:
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(49590, 50) (49590, 50, 19)
(12420, 50) (12420, 50, 19)


In [ ]:
model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=15)

Epoch 1/15
551/551 [==============================] - 14s 11ms/step - loss: 0.1772 - accuracy: 0.9082
Epoch 2/15
551/551 [==============================] - 6s 11ms/step - loss: 0.0463 - accuracy: 0.9686
Epoch 3/15
551/551 [==============================] - 6s 11ms/step - loss: 0.0347 - accuracy: 0.9755
Epoch 4/15
551/551 [==============================] - 6s 11ms/step - loss: 0.0289 - accuracy: 0.9789
Epoch 5/15
551/551 [==============================] - 6s 11ms/step - loss: 0.0250 - accuracy: 0.9814
Epoch 6/15
551/551 [==============================] - 6s 11ms/step - loss: 0.0218 - accuracy: 0.9836
Epoch 7/15
551/551 [==============================] - 6s 11ms/step - loss: 0.0191 - accuracy: 0.9856
Epoch 8/15
551/551 [==============================] - 6s 11ms/step - loss: 0.0166 - accuracy: 0.9874
Epoch 9/15
551/551 [==============================] - 6s 11ms/step - loss: 0.0147 - accuracy: 0.9889
Epoch 10/15
551/551 [==============================] - 6s 11ms/step - loss: 0.0132 - accur

In [ ]:
# batch size in eval
model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE)

138/138 [==============================] - 3s 5ms/step - loss: 0.0937 - accuracy: 0.9620


[0.09373655170202255, 0.9619970917701721]